In [126]:
import numpy as np
from random import random
import matplotlib.pyplot as plt

In [127]:
print(random())

0.5422110108570316


In [128]:
def generateDataSamples(G1,G2,C1,C2,P1,P2,T1,T2):
    sampleSize = 100
    sample=[]
    avg = []
    for i in range(sampleSize):
        G = round(G1 + (G2-G1)*random(),0)/340
        C = round(C1 + (C2-C1)*random(),1)/10.0
        P = round(P1 + (P2-P1)*random(),0)/6.0
        T = round(T1 + (T2-T1)*random(),0)/120.0
        sample.append([G,C,P,T])
        G = round(G1 + (G2-G1)*random(),0)
        C = round(C1 + (C2-C1)*random(),1)
        P = round(P1 + (P2-P1)*random(),0)
        T = round(T1 + (T2-T1)*random(),0)
        avg.append([G,C,P,T])

    return sample,avg




In [132]:
S1,avg = generateDataSamples(320,340,9,10,1,6,100,120)
average = []
for i in range(len(avg[0])):
    average.append(0)

for i in range(len(avg)):
    for j in range(len(avg[0])):
        average[j] += avg[i][j]
for i in range(len(average)):
    average[i] = round(average[i]/len(avg),1)
average[0] = round(average[0],0)
average[2] = round(average[2],0)
average[3] = round(average[3],0)
print(average,"average")
S2,avg = generateDataSamples(280,330,6,9,0,6,80,120)
train = S1
classes = []
for i in range(len(S1)):
    classes.append(1)
for i in range(len(S2)):
    classes.append(0)
for i in range(len(S2)):
    S1.append(S2[i])
train=S1
print(len(train))
#print(train)



[330.0, 9.5, 4.0, 110.0] average
200


In [133]:


l=3 # Choose number of layers here
N = [4,3,1] # No of neurons in each layer
#train = [[9,9,3],[10,10,3],[9,10,3],[10,9,3],[11,11,3],[10,11,3],[11,10,3],[12,9,3],[9,12,3],[14,10,3],[9,13,3],[10,13,3],[11,8,3],[8,12,3]]
#train = [[1,1,3],[-1,-1,3],[-2,-2,3],[-3,-3,3], [2,2,3],[3,3,3],[4,4,3],[5,5,3]]
#classes = [0,0,0,0,0,0,0,1,1,1,1,1,1,1]
#classes = [0,0,0,0,1,1,1,1]

#class1 = [[9,9],[10,10],[9,10],[10,9],[11,11],[10,11],[11,10]]
#class2 = [[12,9],[9,12],[14,10],[9,13],[10,13],[11,8],[8,12]]
#y=[0,0,0,0,1,1,1,1]
"""for i in range(len(train)):
    if i<=6:
        y.append(0)
    else:
        y.append(1)"""

w=[]
b=[]
eta=0.5
#adam optimiser parameters
alpha = 0.001 # learning rate
beta1 = 0.9# parameter for momentum term
beta2 = 0.999 # parameter for rms prop
epsilon = 10**(-8)
Vdw=[]
Vdb=[]
Sdw=[]
Sdb=[]
for i in range(0,l-1): # initialising the weights
    temp = np.random.randn( N[i+1],N[i] )*0.01
    w.append(temp)
    temp = np.random.randn( N[i+1],1 ) * 0.01
    temp = temp.reshape(-1,1)
    b.append(temp)


for i in range(0,l-1): # initilaising the paramteres for adam optimiser
    temp = np.zeros([ N[i+1],N[i] ],dtype=float)
    Vdw.append(temp)
    temp = np.zeros( [N[i+1],1 ],dtype=float)
    temp=temp.reshape(-1,1)
    Vdb.append(temp)
    temp = np.zeros( [ N[i+1],N[i] ],dtype=float )
    Sdw.append(temp)
    temp = np.zeros( [N[i+1],1],dtype=float )
    temp = temp.reshape(-1,1)
    Sdb.append(temp)


trials = 10000 #100000 gives 100 accuracy

for trial in range(trials):
    
    

    changeTemp = []
    changeBTemp = []
    for i in range(0,l-1):
        temp = []
        for i1 in range(N[i+1]):
            temp1 = []
            for i2 in range(N[i]):
                temp1.append(0)
            temp.append(temp1)
        tempB = []
        for i1 in range(0,N[i+1]):
            tempB.append(0)
        
        temp = np.array(temp)
        tempB = np.array(tempB)
        tempB = tempB.reshape(-1,1)
        changeTemp.append(temp)
        changeBTemp.append(tempB)

    
    
    changeMatrix = changeTemp # initialising the gradient matrix for weights
    changeBMatrix = changeBTemp # initialsiing gradient matrix for biases


    for i in range(0,len(train)): # we are considering the entire training set as a batch
        aTemp = []
        deltaTemp = []

        for p in range(0,l):
            tempA = []
            tempDelta = []

            for j in range(0,N[p]):
                tempA.append(0)
                tempDelta.append(0)
            tempA = np.array(tempA)
            tempA = tempA.reshape(-1,1)
            tempDelta = np.array(tempDelta)
            tempDelta = tempDelta.reshape(-1,1)

            aTemp.append(tempA)
            deltaTemp.append(tempDelta)

        a = aTemp
        delta = deltaTemp

        
        
        trainSample = np.array(train[i])
        trainSample = trainSample.reshape(-1,1)
        a[0] = trainSample
        

        
        for j in range(1,l): #forward Prop
            
            z = w[j-1].dot( a[j-1]) + b[j-1]
            a[j] = (1/(1 + np.exp(-z)))
            
            
        if (classes[i] == 0 and a[l-1][0][0] >0.001 ) or (classes[i] == 1 and a[l-1][0][0] <0.999):# checking if there's error
            for j in range(l-1,0,-1):# when there's error, error in every neuron is found using vectorised equations
                if j== l-1:
                    
                    temp = classes[j]
                    temp = np.array(classes[j])
                    temp = temp.reshape(1,1)
                    
                    delta[j] =  (a[j] - classes[i]) * a[j] * (1-a[j])
                    
                    
                else:
                    
                    delta[j] = w[j].transpose().dot( delta[j+1])
                    
                    
                    temp = np.multiply(a[j],1-a[j])
                    delta[j] = np.multiply(delta[j],temp)
    
        
        for j in range(0,l-1):

            
            
            changeMatrix[j] = changeMatrix[j] + delta[j+1].dot( a[j].transpose() ) # udating the gradients of weights here
            changeBMatrix[j] = changeBMatrix[j] + delta[j+1] # updating the gradients of biases here
            
        
    

    for i in range(0,len(w)):

        N1 = 1.0/len(train)
        
        
        changeMatrix[i] = N1*changeMatrix[i]
        changeBMatrix[i] =  N1*changeBMatrix[i]
        
        Vdw[i] = beta1 * Vdw[i] + (1-beta1) * changeMatrix[i]
    
        Vdb[i] = beta1*Vdb[i] + (1-beta1) * changeBMatrix[i]

        
        Sdw[i] = beta2*Sdw[i] + (1-beta2) * np.square(changeMatrix[i])
    

        Sdb[i] = beta2*Sdb[i] + (1-beta2) * np.square(changeBMatrix[i])
        
        
    VdwCorrected = []
    VdbCorrected = []
    SdwCorrected = []
    SdbCorrected = []

    for i in range(0,len(w)):
        

        VdwCorrected.append( Vdw[i]/( 1- beta1**( trial+1 )) )
        
        VdbCorrected.append( Vdb[i]/( 1- (beta1**(trial+1) ) ) )

        SdwCorrected.append( Sdw[i]/( 1- (beta2** (trial+1) ) ) )
        SdbCorrected.append( Sdb[i]/( 1- (beta2** (trial+1) )  ) )

    
    for i in range(0,len(w)):

        w[i] = w[i] - alpha * ( VdwCorrected[i]/ ( np.sqrt(SdwCorrected[i]) + epsilon  ) )
        b[i] = b[i] - alpha * ( VdbCorrected[i]/ ( np.sqrt(SdbCorrected[i]) + epsilon  ) )

    
    if (trial % (trials/100) ) == 0:
        print("Done", trial/(trials/100))








#testing here
prop=0
for i in range(0,len(train)):
    aTemp = []
    deltaTemp = []

    for p in range(0,l):
        tempA = []
        tempDelta = []

        for j in range(0,N[p]):
            tempA.append(0)
            tempDelta.append(0)

        tempA = np.array(tempA)
        tempA = tempA.reshape(-1,1)
        tempDelta = np.array(tempDelta)
        tempDelta = tempDelta.reshape(-1,1)

        aTemp.append(tempA)
        deltaTemp.append(tempDelta)

    a = aTemp
    delta = deltaTemp

    trainSample = np.array(train[i])
    trainSample = trainSample.reshape(-1,1)
    a[0] = trainSample

    for j in range(1,l): #forward Prop
        
        z = w[j-1].dot( a[j-1]) + b[j-1]
        a[j] = (1/(1 + np.exp(-z)))
    
    print(a[l-1][0][0],"val")
    if a[l-1][0][0] <0.5:
        print("Original class",classes[i], "Predicted Class",0)
        
    if a[l-1][0][0] >0.5:
        print("Original class",classes[i], "Predicted Class",1)
    if (classes[i] == 0 and a[l-1][0][0] <0.5 ) or (classes[i] == 1 and a[l-1][0][0] >0.5):
        #print(train[i],classes[i])
        prop+=1
print("Accuracy")
print( (float(prop)/len(train))*100 )

print()









    

Done 0.0
Done 1.0
Done 2.0
Done 3.0
Done 4.0
Done 5.0
Done 6.0
Done 7.0
Done 8.0
Done 9.0
Done 10.0
Done 11.0
Done 12.0
Done 13.0
Done 14.0
Done 15.0
Done 16.0
Done 17.0
Done 18.0
Done 19.0
Done 20.0
Done 21.0
Done 22.0
Done 23.0
Done 24.0
Done 25.0
Done 26.0
Done 27.0
Done 28.0
Done 29.0
Done 30.0
Done 31.0
Done 32.0
Done 33.0
Done 34.0
Done 35.0
Done 36.0
Done 37.0
Done 38.0
Done 39.0
Done 40.0
Done 41.0
Done 42.0
Done 43.0
Done 44.0
Done 45.0
Done 46.0
Done 47.0
Done 48.0
Done 49.0
Done 50.0
Done 51.0
Done 52.0
Done 53.0
Done 54.0
Done 55.0
Done 56.0
Done 57.0
Done 58.0
Done 59.0
Done 60.0
Done 61.0
Done 62.0
Done 63.0
Done 64.0
Done 65.0
Done 66.0
Done 67.0
Done 68.0
Done 69.0
Done 70.0
Done 71.0
Done 72.0
Done 73.0
Done 74.0
Done 75.0
Done 76.0
Done 77.0
Done 78.0
Done 79.0
Done 80.0
Done 81.0
Done 82.0
Done 83.0
Done 84.0
Done 85.0
Done 86.0
Done 87.0
Done 88.0
Done 89.0
Done 90.0
Done 91.0
Done 92.0
Done 93.0
Done 94.0
Done 95.0
Done 96.0
Done 97.0
Done 98.0
Done 99.0
0.88787154

In [135]:
print(w)
print(b)

[array([[ 3.50289036,  6.20797784,  0.03194757,  0.92241371],
       [-3.43750174, -6.25863234, -0.03308682, -0.91633425],
       [-3.51830186, -6.21178742, -0.03175965, -0.92211386]]), array([[ 8.16308201, -8.14460566, -8.16273197]])]
[array([[-9.50996358],
       [ 9.48867028],
       [ 9.52790616]]), array([[3.2986039]])]


In [137]:
"""[array([[-3.43325717, -6.93648135,  0.07977072,  0.27570831],
       [ 3.4276012 ,  6.96695906, -0.08034467, -0.27135935],
       [ 3.43969576,  6.97135119, -0.08066845, -0.27210358]]), array([[-7.71286476,  7.98071688,  7.97181443]])]
[array([[ 9.02020244],
       [-9.04630541],
       [-9.06110283]]), array([[-4.52417621]])]"""

#testing here
prop=0
train = [[330/340.0, 9/10.0, 0/6.0,  100/120.0], [  1/340.0, 8/10.0, 3/6.0, 1/120.0]]
for i in range(0,len(train)):
    aTemp = []
    deltaTemp = []

    for p in range(0,l):
        tempA = []
        tempDelta = []

        for j in range(0,N[p]):
            tempA.append(0)
            tempDelta.append(0)

        tempA = np.array(tempA)
        tempA = tempA.reshape(-1,1)
        tempDelta = np.array(tempDelta)
        tempDelta = tempDelta.reshape(-1,1)

        aTemp.append(tempA)
        deltaTemp.append(tempDelta)

    a = aTemp
    delta = deltaTemp

    trainSample = np.array(train[i])
    trainSample = trainSample.reshape(-1,1)
    a[0] = trainSample

    for j in range(1,l): #forward Prop
        
        z = w[j-1].dot( a[j-1]) + b[j-1]
        a[j] = (1/(1 + np.exp(-z)))
    
    print(a[l-1][0][0],"val")
    if a[l-1][0][0] <0.5:
        print("Original class",classes[i], "Predicted Class",0)
        print(a[l-2][0],a[l-2][1],a[l-2][2])
    if a[l-1][0][0] >0.5:
        print("Original class",classes[i], "Predicted Class",1)
        print(a[l-2][0],a[l-2][1],a[l-2][2])
    if (classes[i] == 0 and a[l-1][0][0] <0.5 ) or (classes[i] == 1 and a[l-1][0][0] >0.5):
        #print(train[i],classes[i])
        prop+=1

print(w)

0.6724950851383608 val
Original class 1 Predicted Class 1
[0.56113231] [0.43927389] [0.43881982]
2.937024500414137e-06 val
Original class 1 Predicted Class 0
[0.01088292] [0.9884274] [0.9892768]
[array([[ 3.50289036,  6.20797784,  0.03194757,  0.92241371],
       [-3.43750174, -6.25863234, -0.03308682, -0.91633425],
       [-3.51830186, -6.21178742, -0.03175965, -0.92211386]]), array([[ 8.16308201, -8.14460566, -8.16273197]])]


[0, 0, 0, 0]
4


TypeError: unsupported operand type(s) for /: 'list' and 'int'